In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import lightgbm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize

train = pd.read_csv('train.csv',sep=';')
columns = ['cash_in_out','display_type','scanner_code_reader','atm_id']
train.drop(columns, inplace=True, axis=1)

# get the labels
y = train.target.values
train.drop(['target'], inplace=True, axis=1)

x = normalize(train.values)
#
# Create training and validation sets
#
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
#
# Create the LightGBM data containers
#

#train_data = lightgbm.Dataset(x, label=y)
#test_data = lightgbm.Dataset(x_test, label=y_test)


#print(train_data.shape, test_data.shape, y.shape)

#categoricals = ['cash_in_out','display_type']
#indexes_of_categories = [train.columns.get_loc(col) for col in categoricals]

gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=x, y=y)

param_grid = {
    #'num_leaves': [31, 127],
    #'reg_alpha': [0.1, 0.5],
    #'min_data_in_leaf': [30, 50, 100, 300, 400],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
    }

#lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='binary', num_boost_round=2000, learning_rate=0.01, metric='auc',categorical_feature=indexes_of_categories)
lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  
                                   objective='binary',  
                                   learning_rate=0.01, 
                                   metric='auc')

gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=gkf)
lgb_model = gsearch.fit(x, y)


print(lgb_model.best_params_, lgb_model.best_score_)